In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-121434")
exp = Experiment(workspace=ws, name="quick-starts-ws-121434")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-121434
Azure region: southcentralus
Subscription id: 94e14ad4-bf97-47e8-aae0-f9b85a7befa8
Resource group: aml-quickstarts-121434


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_configurations = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
compute_cluster = ComputeTarget.create(ws, 'project-cluster', compute_configurations)
compute_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(1, 5),
    '--max_iter': choice(10, 50, 100, 150),
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=3, slack_factor = 0.3)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',
              entry_script='train.py',
              compute_target=compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=6,
                                     max_concurrent_runs=3,
                                     estimator=est)

In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyper_run = exp.submit(config=hyperdrive_config, show_output=True)

In [18]:
RunDetails(hyper_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [24]:
import joblib
# Get your best run and save the model from that run.

best_run = hyper_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

joblib.dump(value=parameter_values, filename='./training/best_hyperdrive_model.joblib')
print(f"accurracy: {best_run_metrics['Accuracy']}")

accurracy: 0.9160849772382398


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

tabular_data = TabularDatasetFactory.from_delimited_files(path, encoding='utf8')

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(tabular_data)
training_data = x.assign(y=y)

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    debug_log='automl_debug.log',
    primary_metric='accuracy',
    training_data=training_data,
    label_column_name='y',
    enable_early_stopping=True,
    n_cross_validations=5)

auto_run = exp.submit(config=automl_config, show_output=False)


In [12]:
# Submit your automl run

RunDetails(auto_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [25]:
# Retrieve and save your best automl model.
metrics, best_model = auto_run.get_output()

joblib.dump(value=best_model, filename='./training/autorun_model.joblib')

['./training/autorun_model.joblib']

In [26]:
compute_cluster.delete()